## What is this notebook for?

We want to create a web interface for our users to chat with a model, the model will have its own instructions.

In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [4]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [5]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

In [6]:
message_gpt("What is today's date?")

"Today's date is October 4, 2023."

In [7]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [8]:
message("hello")

Shout has been called with input hello


'HELLO'

We'll create an interface that uppercases anything we input on it

In [9]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Shout has been called with input 
Shout has been called with input hello
Shout has been called with input teu bulisco
Created dataset file at: .gradio/flagged/dataset1.csv


In [13]:
# Adding share=True means that it can be accessed publically
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://ac87f7b081850f2577.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Using messsage_gpt function to talk with our LLM model, and sharing across the internet.

In [16]:
system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=message_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://3f9ecff3ebaa0a6be7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Now putting the response in chunks to see if it simulates a fluid conversation

In [17]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


## Building a chatbot with Gradio interface and GPT model

Here we build an AI assistant using GPT-4o-mini model, and we maintain an external endpoint for public access using Gradio. The interface will be a little more complex than the one that we use right now.

In [26]:
openai = OpenAI()
MODEL = 'gpt-4o-mini'
system_message = "You are a survey maker, clients will ask you for help to create surveys related to their areas of expertise. They will input pieces of text from \
    some papers. You will help them by making a summary of those pieces of papers and making a section related to a survey section containing relevant information from all those papers. \
    They will inform the area of expertise, the section, and pieces of text from different papers. You will be responsible to make a summary of the papers and provide a latex file section. \
        For example: Area of expertise: Neural Networks for texture classification, Section: Introduction, Pieces of text: Texture is a relevant attribute... . \
            You shall also provide the citations used in bibtex format apart from the tex file."

In [27]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [28]:
gr.ChatInterface(fn=chat, type="messages").launch(pwa=True, share=True)

* Running on local URL:  http://127.0.0.1:7873
* Running on public URL: https://0813e37c61b7bf9b5e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


History is:
[]
And messages is:
[{'role': 'system', 'content': 'You are a survey maker, clients will ask you for help to create surveys related to their areas of expertise. They will input pieces of text from     some papers. You will help them by making a summary of those pieces of papers and making a section related to a survey section summarizing the papers.     They will inform the area of expertise, the section, and pieces of text from different papers. You will be responsible to make a summary of the papers and provide a latex file section.         For example: Area of expertise: Neural Networks for texture classification, Section: Introduction, Pieces of text: Texture is a relevant attribute... .             You shall also provide citations in bibtex format apart from the tex file.'}, {'role': 'user', 'content': 'Area of expertise: Convolutional Neural Networks for Texture Classification\nSection: Introduction\nPieces of Text: \n\nFor several decades, texture has been studied in